Create a DataFrame from a CSV.

In [1]:
val clickstreamDF = sqlContext.read.format("com.databricks.spark.csv")
  .option("header", "true")
  .option("delimiter", ",")
  .option("mode", "PERMISSIVE")
  .option("inferSchema", "true")
  .load("file:///mnt/ephemeral/summitdata/MusicCSV_tracks_by_album.csv")

Count the rows in the DataFrame

In [2]:
clickstreamDF.count()

Long = 36737

Let's take a look at the schema for the DataFrame

In [3]:
clickstreamDF.printSchema()

root
 |-- album: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- performer: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- number: integer (nullable = true)
 |-- title: string (nullable = true)



Rename DataFrame columns to those of the Cassandra table.

In [10]:
val renamed = clickstreamDF
                .withColumnRenamed("album", "album_title")
                .withColumnRenamed("year", "album_year")
                .withColumnRenamed("genre", "album_genre")
                .withColumnRenamed("number", "track_number")
                .withColumnRenamed("title", "track_title")
                

Write the DataFrame to an already-created table.

In [11]:
renamed.write.format("org.apache.spark.sql.cassandra").options(Map("table" -> "tracks_by_album", "keyspace" -> "musiccsv")).save()

Check to see that we inserted them all

In [12]:
%%cql select count(*) from musiccsv.tracks_by_album;

count
36737
